In [1]:
from nnsight import LanguageModel

import gc
import itertools
import math
import os
import random
import sys
from collections import Counter
from copy import deepcopy
from dataclasses import dataclass
from functools import partial
from pathlib import Path
from typing import Any, Callable, Literal, TypeAlias

import einops
import numpy as np
import pandas as pd
import plotly.express as px
import requests
import torch as t
from datasets import load_dataset
from huggingface_hub import hf_hub_download
from IPython.display import HTML, IFrame, clear_output, display
from jaxtyping import Float, Int
from rich import print as rprint
from rich.table import Table
from sae_lens import (
    SAE,
    ActivationsStore,
    HookedSAETransformer,
    LanguageModelSAERunnerConfig,
    SAEConfig,
    SAETrainingRunner,
    upload_saes_to_huggingface,
)
from sae_lens.toolkit.pretrained_saes_directory import get_pretrained_saes_directory
from sae_vis import SaeVisConfig, SaeVisData, SaeVisLayoutConfig
from tabulate import tabulate
from torch import Tensor, nn
from torch.distributions.categorical import Categorical
from torch.nn import functional as F
from tqdm.auto import tqdm
from transformer_lens import ActivationCache, HookedTransformer, utils
from transformer_lens.hook_points import HookPoint

device = "cuda" if t.cuda.is_available() else "mps" if t.backends.mps.is_available() else "cpu"

/n/data2/hms/dbmi/sunyaev/lab/dlee/.cache/pypoetry/virtualenvs/refusal-direction-f5Ymycjl-py3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
## check memory usage

if t.cuda.is_available():
    gpu_id = 0  # Set to your target GPU ID
    total_memory = t.cuda.get_device_properties(gpu_id).total_memory
    allocated_memory = t.cuda.memory_allocated(gpu_id)
    cached_memory = t.cuda.memory_reserved(gpu_id)

    print(f"Total GPU Memory: {total_memory / 1024**2:.2f} MB")
    print(f"Allocated GPU Memory: {allocated_memory / 1024**2:.2f} MB")
    print(f"Cached GPU Memory: {cached_memory / 1024**2:.2f} MB")
elif t.backends.mps.is_available():
    # MPS (Metal Performance Shaders) for Mac
    print("MPS is available.")
    # Note: As of now, PyTorch doesn't provide direct memory management functions for MPS
    print("Memory information is not available for MPS.")
else:
    print("Neither CUDA nor MPS is available.")

Total GPU Memory: 45541.31 MB
Allocated GPU Memory: 0.00 MB
Cached GPU Memory: 0.00 MB


In [3]:
# del gemma2
# del gemma2_sae

t.cuda.empty_cache()


# Output tensor of SAE activations for advbench

In [4]:
import json

# Read from advbench.json file
with open('../dataset/processed/advbench.json', 'r') as file:
    advbench_data = json.load(file)

len(advbench_data)

# Read from advbench.json file
with open('../dataset/processed/alpaca.json', 'r') as file:
    alpaca_data = json.load(file)

print(len(alpaca_data))

31323


In [5]:
alpaca_data_500 = alpaca_data[:500]
alpaca_data_10000 = alpaca_data[:10000]

In [6]:
len(alpaca_data_10000)

10000

In [7]:
del stacked_sae_acts_post
del gemma2_sae
t.cuda.empty_cache()


NameError: name 'stacked_sae_acts_post' is not defined

In [8]:
def save_sae_activations(sae_name, sae_ids, data, suffix):
    t.set_grad_enabled(False)
    gemma2: HookedSAETransformer = HookedSAETransformer.from_pretrained("gemma-2-2b-it", device=device)

    for sae_id in sae_ids:
        gemma2_sae, cfg_dict, sparsity = SAE.from_pretrained(
            release=sae_name,
            sae_id=sae_id,
            device=str(device),
        )
    
        all_sae_acts_post = []
    
        for item in data:
            prompt = item['instruction']
            
            # Get top activations on final token
            _, cache = gemma2.run_with_cache_with_saes(
                prompt,
                saes=[gemma2_sae],
                stop_at_layer=gemma2_sae.cfg.hook_layer + 1,
            )
            sae_acts_post = cache[f"{gemma2_sae.cfg.hook_name}.hook_sae_acts_post"][0, -1, :]
            all_sae_acts_post.append(sae_acts_post)
    
        # Stack all sae_acts_post tensors
        stacked_sae_acts_post = torch.stack(all_sae_acts_post)
    
        print(f"SAE ID: {sae_id}")
        print(f"Shape of stacked tensor: {stacked_sae_acts_post.shape}")
        print(f"Total number of non-zero activations: {(stacked_sae_acts_post != 0).sum().item()}")
    
        # Create directory if it doesn't exist
        os.makedirs(f'../data/sae_acts/{sae_name}/{sae_id}', exist_ok=True)
    
        # Save the stacked_sae_acts_post tensor
        torch.save(stacked_sae_acts_post.cpu(), f'../data/sae_acts/{sae_name}/{sae_id}_{suffix}.pt')
    
        # Print confirmation message
        print(f"Stacked SAE activations saved to '../data/sae_acts/{sae_name}/{sae_id}_{suffix}.pt'")
        print("---")
    
        del stacked_sae_acts_post
        del gemma2_sae
        t.cuda.empty_cache()
    

In [9]:
sae_name = "gemma-scope-2b-pt-res-canonical"
# sae_ids = [f"layer_{layer}/width_16k/canonical" for layer in range(11)]  # 0 through 10
# sae_ids = [f"layer_{layer}/width_16k/canonical" for layer in [5]]  # 0 through 10
sae_ids = [f"layer_{layer}/width_16k/canonical" for layer in range(5)]  # 0 through 10

save_sae_activations(sae_name, sae_ids, alpaca_data_10000, suffix = "alpaca_10000")

Loading checkpoint shards:   0%|                                                                                                                                   | 0/2 [00:05<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
gemma2: HookedSAETransformer = HookedSAETransformer.from_pretrained("gemma-2-2b-it", device=device)

for sae_id in sae_ids:
    gemma2_sae, cfg_dict, sparsity = SAE.from_pretrained(
        release=sae_name,
        sae_id=sae_id,
        device=str(device),
    )

    all_sae_acts_post = []

    for item in advbench_data:
        prompt = item['instruction']
        
        # Get top activations on final token
        _, cache = gemma2.run_with_cache_with_saes(
            prompt,
            saes=[gemma2_sae],
            stop_at_layer=gemma2_sae.cfg.hook_layer + 1,
        )
        sae_acts_post = cache[f"{gemma2_sae.cfg.hook_name}.hook_sae_acts_post"][0, -1, :]
        all_sae_acts_post.append(sae_acts_post)

    # Stack all sae_acts_post tensors
    stacked_sae_acts_post = torch.stack(all_sae_acts_post)

    print(f"SAE ID: {sae_id}")
    print(f"Shape of stacked tensor: {stacked_sae_acts_post.shape}")
    print(f"Total number of non-zero activations: {(stacked_sae_acts_post != 0).sum().item()}")

    # Create directory if it doesn't exist
    os.makedirs(f'../data/sae_acts/{sae_name}/{sae_id}', exist_ok=True)

    # Save the stacked_sae_acts_post tensor
    torch.save(stacked_sae_acts_post.cpu(), f'../data/sae_acts/{sae_name}/{sae_id}_advbench.pt')

    # Print confirmation message
    print(f"Stacked SAE activations saved to '../data/sae_acts/{sae_name}/{sae_id}_advbench.pt'")
    print("---")

    del stacked_sae_acts_post
    del gemma2_sae
    t.cuda.empty_cache()

Loading checkpoint shards: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:08<00:00,  4.25s/it]


Loaded pretrained model gemma-2-2b-it into HookedTransformer
SAE ID: layer_0/width_16k/canonical
Shape of stacked tensor: torch.Size([520, 16384])
Total number of non-zero activations: 67218
Stacked SAE activations saved to '../data/sae_acts/gemma-scope-2b-pt-res-canonical/layer_0/width_16k/canonical_advbench.pt'
---
SAE ID: layer_1/width_16k/canonical
Shape of stacked tensor: torch.Size([520, 16384])
Total number of non-zero activations: 64476
Stacked SAE activations saved to '../data/sae_acts/gemma-scope-2b-pt-res-canonical/layer_1/width_16k/canonical_advbench.pt'
---
SAE ID: layer_2/width_16k/canonical
Shape of stacked tensor: torch.Size([520, 16384])
Total number of non-zero activations: 94561
Stacked SAE activations saved to '../data/sae_acts/gemma-scope-2b-pt-res-canonical/layer_2/width_16k/canonical_advbench.pt'
---
SAE ID: layer_3/width_16k/canonical
Shape of stacked tensor: torch.Size([520, 16384])
Total number of non-zero activations: 44623
Stacked SAE activations saved to '.

# get average activations for each sae

In [4]:
t.set_grad_enabled(False)

gpt2: HookedSAETransformer = HookedSAETransformer.from_pretrained("gpt2-small", device=device)

gpt2_sae, cfg_dict, sparsity = SAE.from_pretrained(
    release="gpt2-small-res-jb",
    sae_id="blocks.7.hook_resid_pre",
    device=str(device),
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Loaded pretrained model gpt2-small into HookedTransformer


/n/data2/hms/dbmi/sunyaev/lab/dlee/.cache/pypoetry/virtualenvs/refusal-direction-f5Ymycjl-py3.12/lib/python3.12/site-packages/sae_lens/sae.py:143: UserWarning: 
This SAE has non-empty model_from_pretrained_kwargs. 
For optimal performance, load the model like so:
model = HookedSAETransformer.from_pretrained_no_processing(..., **cfg.model_from_pretrained_kwargs)
  warnings.warn(


In [5]:
gpt2_act_store = ActivationsStore.from_sae(
    model=gpt2,
    sae=gpt2_sae,
    streaming=True,
    store_batch_size_prompts=16,
    n_batches_in_buffer=32,
    device=str(device),
)

# Example of how you can use this:
tokens = gpt2_act_store.get_batch_tokens()
assert tokens.shape == (gpt2_act_store.store_batch_size_prompts, gpt2_act_store.context_size)

/n/data2/hms/dbmi/sunyaev/lab/dlee/.cache/pypoetry/virtualenvs/refusal-direction-f5Ymycjl-py3.12/lib/python3.12/site-packages/sae_lens/training/activations_store.py:246: UserWarning: Dataset is not tokenized. Pre-tokenizing will improve performance and allows for more control over special tokens. See https://jbloomaus.github.io/SAELens/training_saes/#pretokenizing-datasets for more info.
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (1217 > 1024). Running this sequence through the model will result in indexing errors


In [7]:
def get_frac_active(
    model: HookedSAETransformer,
    sae: SAE,
    act_store: ActivationsStore,
    # latent_idx: int,
    total_batches: int = 400,
):
    """
    Displays the activation histogram for a particular latent, computed across `total_batches` batches from `act_store`.
    """
    sae_acts_post_hook_name = f"{sae.cfg.hook_name}.hook_sae_acts_post"
    all_positive_acts = []

    all_positive_acts = t.zeros(sae.cfg.d_sae, device=device)
    total_acts = 0

    for i in tqdm(range(total_batches)):
        tokens = act_store.get_batch_tokens()
        _, cache = model.run_with_cache_with_saes(
            tokens,
            saes=[sae],
            stop_at_layer=sae.cfg.hook_layer + 1,
            names_filter=[sae_acts_post_hook_name],
        )
        acts = cache[sae_acts_post_hook_name]
        all_positive_acts += (acts > 0).sum(dim=(0, 1))
        total_acts += acts.shape[0] * acts.shape[1]

    frac_active = all_positive_acts / total_acts

    return frac_active


frac_active = get_frac_active(gpt2, gpt2_sae, gpt2_act_store)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 400/400 [00:25<00:00, 15.64it/s]


In [9]:
frac_active[3731]

tensor(0.1123, device='cuda:0')

In [ ]:
t.set_grad_enabled(False)

sae_name = "gemma-scope-2b-pt-res-canonical"
sae_ids = [f"layer_{layer}/width_16k/canonical" for layer in range(11)]  # 0 through 10

gemma2: HookedSAETransformer = HookedSAETransformer.from_pretrained("gemma-2-2b-it", device=device)

In [ ]:
for sae_id in sae_ids:
    gemma2_sae, cfg_dict, sparsity = SAE.from_pretrained(
        release=sae_name,
        sae_id=sae_id,
        device=str(device),
    )

    gemma2_act_store = ActivationsStore.from_sae(
        model=gemma2,
        sae=gemma2_sae,
        streaming=True,
        store_batch_size_prompts=16,
        n_batches_in_buffer=32,
        device=str(device),
    )

    frac_active = get_frac_active(gemma2, gemma2_sae, gemma2_act_store)

    # Create directory if it doesn't exist
    os.makedirs(f'../data/sae_acts/{sae_name}/{sae_id}', exist_ok=True)

    # Save the stacked_sae_acts_post tensor
    t.save(frac_active, f'../data/sae_acts/{sae_name}/{sae_id}/frac_active.pt')
    print(f"SAE ID: {sae_id}")
    print(f"Fraction of activations: {frac_active}")

    del gemma2_sae
    t.cuda.empty_cache()

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 400/400 [03:40<00:00,  1.82it/s]


SAE ID: layer_0/width_16k/canonical
Fraction of activations: tensor([0.0323, 0.0013, 0.0100,  ..., 0.0024, 0.0030, 0.0010], device='cuda:0')


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 400/400 [05:25<00:00,  1.23it/s]


SAE ID: layer_1/width_16k/canonical
Fraction of activations: tensor([0.0011, 0.0016, 0.0276,  ..., 0.0004, 0.0009, 0.0008], device='cuda:0')


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 400/400 [07:08<00:00,  1.07s/it]


...

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 400/400 [15:48<00:00,  2.37s/it]


SAE ID: layer_7/width_16k/canonical
Fraction of activations: tensor([0.0008, 0.0101, 0.0005,  ..., 0.0010, 0.0026, 0.0043], device='cuda:0')


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 400/400 [17:32<00:00,  2.63s/it]


SAE ID: layer_8/width_16k/canonical
Fraction of activations: tensor([0.0034, 0.0097, 0.0066,  ..., 0.0054, 0.0058, 0.0058], device='cuda:0')


 30%|████████████████████████████████████████████▌                                                                                                     | 122/400 [05:53<13:28,  2.91s/it]

In [18]:
del gemma2_sae
t.cuda.empty_cache()